In [1]:
import torch
import numpy as np
from torch import nn
import random
import torch.nn.functional as F
import collections
from torch.optim.lr_scheduler import StepLR
import time
import shelve
import os
from datetime import datetime
from matplotlib import pyplot as plt
import pickle
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from tqdm.notebook import tqdm


# get hists by method
workspace_folder_path = '/media/nero-ia/7A309A87309A49D1/sia_23/25k'

#'E:/sia_23/25k'
#'C:/Users/alexa/Desktop/sia_DRL_2023/25k/'
#  '/media/xnd/7A309A87309A49D1/sia_23/25k/'

def get_complete_hist_from_shelve(workspace_folder_path, agg_method, n_sectors):
    myshelve = shelve.open(workspace_folder_path+'/wsh_{metodo}{n_setores}completo.out'.format(
        metodo = agg_method,n_setores = n_sectors
    ))

    hist_dict = myshelve['hist_dict']
    myshelve.close()
    return hist_dict

training_list = [
    ['mean','4']
    ,['mean','5']
    ,['mean','6']
    ,['mean','10']
    ,['min','4']
    ,['min','5']
    ,['min','6']
    ,['min','10']
    ,['mode','4']
    ,['mode','5']
    ,['mode','6']
    ,['mode','10']
    ]

training_res_dict = {}
for agg_method, n_sectors in tqdm(training_list):
    print('Processing: {agg_method}-{n_sectors}'.format(agg_method=agg_method,  n_sectors=n_sectors))
    hist = get_complete_hist_from_shelve(workspace_folder_path, agg_method, n_sectors)

    training_res_dict[agg_method+'_'+n_sectors] = hist

    del hist

import pickle
with open(workspace_folder_path+'training_res_dict.pickle', 'wb') as handle:
    pickle.dump(training_res_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

  0%|          | 0/12 [00:00<?, ?it/s]

Processing: mean-4
Processing: mean-5
Processing: mean-6
Processing: mean-10


: 

In [ ]:
# metrics
def calc_vec_tdt(qn_test_results_dict, env_and_method:str = 'goal_0mean_4'):
  tdt_ep = []
  pos =qn_test_results_dict[env_and_method]['pos']

  # Extract position from each episode and sum tdt
  for ep in range(1,len(pos)+1):
    tdt = 0
    for n_iter in range(0, len(pos[ep])):
      # last position:
      x0 =pos[ep][n_iter-1][0]
      y0 =pos[ep][n_iter-1][1]


      # current position:
      y1 =pos[ep][n_iter][1]
      x1 =pos[ep][n_iter][0]

      # distance traveled
      tdt = np.sqrt((x1-x0)**2 +(y1-y0)**2 ) + tdt

    # concatenate tdt for each episode
    tdt_ep.append(tdt)

  return tdt_ep

def calc_vec_fdtg(qn_test_results_dict, env_and_method:str = 'goal_0mean_4'):
  current_goal = int(env_and_method.split('_')[1][0])
  goal = {0:[-8,8],
              1:[-2,8],
              2:[8,-3],
              3:[-8,-8],
              4:[8,8]}[current_goal]

  fdtg_ep = []
  pos = qn_test_results_dict[env_and_method]['pos']

  # Extract position from each episode and append fdtg
  for ep in range(1,len(pos)+1):
    xf = pos[ep][len(pos[ep])-1][0]
    yf = pos[ep][len(pos[ep])-1][1]

    fdtg_ep.append(np.sqrt((xf-goal[0])**2 +(yf-goal[1])**2 ))

  return fdtg_ep

def calc_vec_mdto(qn_test_results_dict, env_and_method:str = 'goal_0mean_4'):

  mdto_ep = []
  scans = qn_test_results_dict[env_and_method]['scan']

  # Extract min of scan from each episode and append over iterations
  for ep in range(1,len(scans)+1):
    min_dto = []
    for n_iter in range(0, len(scans[ep])):
      min_dto.append(np.min(scans[ep][n_iter]))

    mdto_ep.append(np.mean(min_dto)) # append mean value of min_dto

  return mdto_ep


# 0 for crash, 1 for goal 2 for time
def get_success_df(qn_test_results_dict, env_and_method:str = 'goal_0mean_4'):
  hist_dict = qn_test_results[env_and_method]
  success = []
  episodes = []
  for ep in hist_dict['epresult'].keys():
      episodes.append(ep)
      if len(hist_dict['epresult'][ep])<=2:
        res =hist_dict['epresult'][ep][1]
      else:
        res = hist_dict['epresult'][ep][-1][1]

      success.append(res)
  df = pd.DataFrame({'Episodes':episodes, 'Success'+env_and_method:success})
  df['Success'+env_and_method] = np.where(df['Success'+env_and_method]==1,1,0)
  return df

def get_collision_df(qn_test_results_dict, env_and_method:str = 'goal_0mean_4'):
  hist_dict = qn_test_results[env_and_method]
  collision = []
  episodes = []
  for ep in hist_dict['epresult'].keys():
      episodes.append(ep)
      if len(hist_dict['epresult'][ep])<=2:
        res =hist_dict['epresult'][ep][1]
      else:
        res = hist_dict['epresult'][ep][-1][1]

      collision.append(res)

  df = pd.DataFrame({'Episodes':episodes, 'Collision'+env_and_method:collision})
  df['Collision'+env_and_method] = np.where(df['Collision'+env_and_method]==0,1,0)
  return df

In [ ]:
qn_test_metrics = {}

for env_and_method in tqdm(training_res_dict.keys()):

  qn_test_metrics[env_and_method] = {
      'tdt'  : calc_vec_tdt(training_res_dict,env_and_method),
      'fdtg' : calc_vec_fdtg(training_res_dict,env_and_method),
      'mdto' : calc_vec_mdto(training_res_dict,env_and_method),
      'sr_df': get_success_df(training_res_dict,env_and_method),
      'cr_df': get_collision_df(training_res_dict,env_and_method)
  }

import pickle
with open(workspace_folder_path+'training_metrics_dict.pickle', 'wb') as handle:
    pickle.dump(training_res_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)